In [ ]:
import pandas as pd
import numpy as np
import pickle
from ydata_profiling import ProfileReport

In [ ]:
pd.set_option('display.max_columns', None) # show all columns in a df

In [ ]:
X_train = pd.read_csv("split_data/train_features.csv")
y_train = pd.read_csv("split_data/train_target.csv")

In [ ]:
df_train = pd.concat([X_train, y_train], axis = 1)

In [ ]:
df_train.shape

In [ ]:
# df_train['row_prop_missing'] = df_train.isna().mean(axis=1) # Not ok to already calculate here - some are missing by design (e.g. sticker)

In [ ]:
df_train.head()

In [ ]:
profile = ProfileReport(df_train)

In [ ]:
profile.to_notebook_iframe()

Observations  / remarks:
- id: make sure not to include
- is_appartment: multicollinearity with subtype -> don't include both
- area: high correlation with price! impute missing values based on median per subtype, add column to indicate original was missing
- added_time: number of minutes / seconds since the property was added? Can be relevant (very expensive properties might take longer before being sold)
- bedrooms: high correlation with area -> price!
- new_building: no remarks
- postcode: no remarks
- lat / lon: impute missings? calculate distance to railway track, airport, highway (noise)?
- advertiser: impute missings or treat as separate category? > 2000 unique values: reduce? 
- foto_amount: no remarks
- is_promoted: constant -> ignore
- subtype: limited number of missings, can be partly imputed based on is_appartment
- sticker (new / price drop): only price drop could be relevant for price? recode missing values to third category 'no sticker'? ! perfect correlation with energy_value?
- price drop date: relevance?
- energy_value: many missings! can be partly imputed using new_building and/or energy_label? note: a missing energy value usually means it's a bad one
- energy_label: many missings! multicollinearity with energy_value - value is more precise 
- province: maybe cross-check with postal code (DQ); use statbel average price per house / appartment data?
- price: maximum 999999?
- Outliers: Treat or not? DT-based methods can handle them

## Rows with many NAs

In [ ]:
# display(df_train[df_train['row_prop_missing'] > .10])

## Duplicate rows

In [ ]:
# Check done: row with most recent (max) added_time does not necessarily correspond to row with lowest price
# For consistency, deduplicate by keeping row with lowest price (assumed to actually also be most recently updated) - this of course won't work in the test set
# Use added_time

# df_deduplicate = df_train.groupby(["area", "bedrooms", "postcode", "lat", "lon", "advertiser", "subtype"], dropna = False).agg({'price': ['nunique','min'], 'added_time': 'max'}).reset_index()
# df_deduplicate.columns = list(map(''.join, df_deduplicate.columns.values))
# df_deduplicate = df_deduplicate.rename(columns = {'pricemin' : 'price'})
# df_deduplicate.head()

df_deduplicate = df_train.groupby(["area", "bedrooms", "postcode", "lat", "lon", "advertiser", "subtype"], dropna = False).agg({'added_time': 'max', 'id': 'nunique'}).reset_index()
df_deduplicate = df_deduplicate.rename(columns = {"id" : "row_count"})
df_deduplicate.head()

In [ ]:
df_train = pd.merge(df_train, df_deduplicate, how = 'inner', on = ["area", "bedrooms", "postcode", "lat", "lon", "advertiser", "subtype", "added_time"])
df_train = df_train.sort_values(by='added_time', ascending=False)
df_train['row_num'] = df_train.groupby(["area", "bedrooms", "postcode", "lat", "lon", "advertiser", "subtype"], dropna = False).cumcount() + 1
df_train = df_train[df_train['row_num'] == 1].drop(columns='row_num')
display(df_train)

## Subtype

Observations / remarks:
- Missing values are partly houses and partly appartments -> can be assigned accordingly
- There are synonyms -> can be grouped together
- 'Andere' is never an appartment
- Group infrequent levels together? Reduce dimensionality (but: not really necessary for DT-based approach?)
- Perform clustering a.o. on price to reduce number of levels?

In [ ]:
pd.crosstab(df_train['subtype'], df_train['is_appartment'], dropna=False)

In [ ]:
subtype_counts = df_train['subtype'].value_counts()
min_subtype_count = 10
map_infrequent_subtypes = subtype_counts[subtype_counts < min_subtype_count].index

df_train['subtype_regrouped'] = df_train['subtype'].apply(lambda x: 'Andere' if x in map_infrequent_subtypes else x) # remove this step?

In [ ]:
with open('intermediate_data/map_infrequent_subtypes.pkl', 'wb') as file:
    pickle.dump(map_infrequent_subtypes, file)

In [ ]:
map_synonyms = {
    'Assistentie-appartement': 'Serviceflat',
    'Villa-landhuis': 'Villa',
    'Moderne villa': 'Villa',
    'Eengezinswoning': 'Woning',    
    # 'Herenwoning': 'Herenhuis',
    # 'Dakappartement': 'Penthouse',
    'Studio met slaaphoek': 'Studio',

    # 'Rijwoning': 'Woning',
    'Gelijkvloers app.': 'Appartement',
    'Uitzonderlijke woning': 'Villa',
    'Herenwoning': 'Villa',
    'Herenhuis': 'Villa',
    'Burgerswoning': 'Woning',
    'Koppelwoning': 'Woning',
    'Duplex': 'Appartement',
    'Triplex': 'Appartement',
    'Bungalow': 'Woning',
    'Hoeve': 'Villa',
    'Fermette': 'Woning',
    'Bel-étage': 'Woning',
    'Hoekwoning': 'Woning',
    'Pastorijwoning': 'Woning',
    'Arbeiderswoning': 'Woning',
    'Loft': 'Loft Penthouse',
    'Dakappartement': 'Appartement',
    'Penthouse': 'Loft Penthouse',
    'Chalet': 'Andere',
    'Cottage': 'Andere',
    'Vakantiewoning': 'Andere'
    
    
}

df_train['subtype_regrouped'] = df_train['subtype_regrouped'].replace(map_synonyms)

df_train['subtype_regrouped'].value_counts()

## Area

Impute area with median value for subtype (regrouped) and province

In [ ]:
df_train['area_missing'] = df_train['area'].isna().astype(int)

In [ ]:
df_train.groupby('is_appartment')['area'].median()

In [ ]:
df_train.groupby('is_appartment')['area'].mean()

In [ ]:
df_train.groupby('is_appartment')['area'].median()

In [ ]:
df_train.groupby('is_appartment')['area'].max()

In [ ]:
median_area = df_train.groupby(['subtype_regrouped', 'province'])['area'].median()

median_area.to_pickle('intermediate_data/median_area.pkl')

In [ ]:
df_train['area_imputed'] = df_train.apply(
    lambda row: median_area.get((row['subtype_regrouped'], row['province']), row['area']) if pd.isna(row['area']) else row['area'],
    axis=1
)

In [ ]:
display(df_train[0:10])

In [ ]:
df_train['area_rel_to_bedrooms'] = df_train['area_imputed'] / (df_train['bedrooms'] + 1)

## Energy value

In [ ]:
df_train['energy_label'].value_counts(dropna = False)

New buildings with energy label d, e, f, g -> probably bad DQ?

In [ ]:
pd.crosstab(df_train['energy_label'], df_train['new_building'], dropna=False)

In [ ]:
# Keep only first letter of energy label, except for 'a+'

df_train['energy_label_regrouped'] = df_train['energy_label'].apply(lambda x: x[0] if isinstance(x, str) and x != 'a+' and x != 'a+' else x)

In [ ]:
pd.crosstab(df_train['energy_label_regrouped'], df_train['new_building'], dropna=False)

For f and g not ok: normally label A corresponds to values 0-100, B to 101-200, etc.

In [ ]:
df_train.groupby(['energy_label_regrouped'])['energy_value'].median()

In [ ]:
df_train.groupby(['energy_label_regrouped', 'new_building'])['energy_value'].median()

In [ ]:
df_train[df_train['energy_value'].isna()]

In [ ]:
pd.crosstab(df_train[df_train['energy_value'].isna()]['energy_label_regrouped'], df_train[df_train['energy_value'].isna()]['new_building'], dropna=False)

For categories with missings, the sample size with valid energy values is large enough (see higher), so it's okay to also impute based on new_building

In [ ]:
median_energy = df_train.groupby(['energy_label_regrouped', 'new_building', 'subtype_regrouped'])['energy_value'].median()

median_energy.to_pickle('intermediate_data/median_energy.pkl')

In [ ]:
df_train['energy_value_missing'] = df_train['energy_value'].isna().astype(int)

In [ ]:
df_train['energy_value_imputed'] = df_train.apply(
    lambda row: median_energy.get((row['energy_label_regrouped'], row['new_building'], row['subtype_regrouped']), row['energy_value']) if pd.isna(row['energy_value']) else row['energy_value'],
    axis=1
)

In [ ]:
df_train['energy_value'].value_counts(dropna = False)

In [ ]:
# Now all rows except for the ones where the energy label is also missing have an energy value

df_train['energy_value_imputed'].value_counts(dropna = False)

In [ ]:
df_train.groupby(['subtype_regrouped', 'new_building'])['energy_value'].median()

In [ ]:
median_energy_wo_label = df_train.groupby(['new_building', 'subtype_regrouped'])['energy_value'].median()

median_energy_wo_label.to_pickle('intermediate_data/median_energy_wo_label.pkl')

In [ ]:
df_train['energy_value_imputed'] = df_train.apply(
    lambda row: median_energy_wo_label.get((row['new_building'], row['subtype_regrouped']), row['energy_value']) if pd.isna(row['energy_value_imputed']) else row['energy_value_imputed'],
    axis=1
)

In [ ]:
# All NAs imputed

df_train['energy_value_imputed'].value_counts(dropna = False)

## Advertiser

High number of categories - apply something similar to Weights Of Evidence (= for classification problems)

Note: some advertisers occur only once and seem to have a person's name (not a real estate agency) -> also informative

Make bins of advertisers based on how often they occur (only once (category 5: person) vs. more (categories 1-4: agency)) and their median price.
Category 5: only occurs once, so no relevant information on 'typical' pricing; this advertiser will (normally) not occur in the train set either 
Catgories 1-4: occurs more than once, categorize based on median price 

In [ ]:
df_train['advertiser'].nunique()

In [ ]:
df_train['advertiser'].value_counts(dropna = False).head(50) 

In [ ]:
# advertiser_counts = df_train['advertiser'].value_counts()
# min_advertiser_count = 10
# map_infrequent_advertisers = advertiser_counts[advertiser_counts < min_advertiser_count].index
# df_train['advertiser_regrouped'] = df_train['advertiser'].apply(lambda x: 'Andere' if x in map_infrequent_advertisers or pd.isna(x) else x)

In [ ]:
median_price_advertiser = df_train.groupby('advertiser', dropna = False).agg(
    advertiser_median_price=('price', 'median'),  
    advertiser_count=('advertiser', 'size')     
).reset_index()

median_price_advertiser.columns = ['advertiser', 'advertiser_median_price', 'advertiser_count']
display(median_price_advertiser)

In [ ]:
median_price_advertiser['advertiser_bin'] = np.where((median_price_advertiser['advertiser_count'] == 1) | (pd.isna(median_price_advertiser['advertiser']))
                                                     , 5, np.nan)
display(median_price_advertiser)

In [ ]:
num_bins = 4  
median_price_advertiser.loc[(median_price_advertiser['advertiser_count'] > 1) & (~pd.isna(median_price_advertiser['advertiser'])), 'advertiser_bin'] = pd.qcut(
    median_price_advertiser.loc[(median_price_advertiser['advertiser_count'] > 1) & ~ (pd.isna(median_price_advertiser['advertiser'])), 'advertiser_median_price'], 
    q=num_bins, 
    labels=False
)
display(median_price_advertiser)

In [ ]:
median_price_advertiser.groupby(['advertiser_bin'])['advertiser_median_price'].median()

In [ ]:
with open('intermediate_data/median_price_advertiser.pkl', 'wb') as file:
    pickle.dump(median_price_advertiser, file)

In [ ]:
df_train = pd.merge(df_train, median_price_advertiser, how='left', on = "advertiser")

In [ ]:
df_train.head()

In [ ]:
df_train['advertiser_bin'].value_counts(dropna = False)

In [ ]:
median_price_per_advertiser_bin = df_train.groupby(['advertiser_bin'], as_index = False)['price'].median()
median_price_per_advertiser_bin.columns = ['advertiser_bin', 'median_price_advertiser_bin']
median_price_per_advertiser_bin

In [ ]:
median_price_per_advertiser_bin.to_pickle('intermediate_data/median_price_per_advertiser_bin.pkl')

In [ ]:
df_train = pd.merge(df_train, median_price_per_advertiser_bin, how='left', on='advertiser_bin')
df_train.head()

## Regional prices - Statbel

In [ ]:
df_map_nis = pd.read_csv("external_data/cities.csv")

In [ ]:
display(df_map_nis)

In [ ]:
df_map_nis['zipCode'].value_counts()

In [ ]:
# if there are multiple lines for the same zip code, there is 1 'main'
df_map_nis[df_map_nis['zipCode'] == 3700]

In [ ]:
df_map_nis[df_map_nis['zipCode'] == 2260]

In [ ]:
df_map_nis[df_map_nis['zipCode'] == 7700]

In [ ]:
df_map_nis[df_map_nis['zipCode'] == 2260]

In [ ]:
df_map_nis_renamed = df_map_nis.rename(columns={'name': 'municipality', 'province': 'province_nis', 'zipCode': 'zip_code', 'nisCode': 'nis_code'}).copy()

In [ ]:
df_map_nis_deduplicated = df_map_nis_renamed.sort_values(by='main', ascending=False).groupby('zip_code').first().reset_index().drop(columns = 'main').copy()

In [ ]:
df_map_nis_deduplicated[df_map_nis_deduplicated['zip_code'] == 3700]

In [ ]:
df_map_nis_deduplicated['zip_code'].value_counts()

In [ ]:
with open('intermediate_data/map_nis_deduplicated.pkl', 'wb') as file:
    pickle.dump(df_map_nis_deduplicated, file)

In [ ]:
df_train = pd.merge(df_train, df_map_nis_deduplicated, how='left', left_on='postcode', right_on = 'zip_code')
display(df_train)

In [ ]:
df_train.shape

In [ ]:
pd.crosstab(df_train['province'], df_train['province_nis'], dropna=False) # Looks okay except for Antwerpen-Oost-Vlaanderen?

In [ ]:
df_statbel_prices = pd.read_excel("external_data/vastgoed_2010_9999.xlsx", sheet_name = '2024')

In [ ]:
display(df_statbel_prices)

In [ ]:
df_statbel_prices['CD_TYPE_NL'].unique()

In [ ]:
df_statbel_prices['CD_CLASS_SURFACE'].unique()

In [ ]:
df_statbel_prices_houses = df_statbel_prices[(df_statbel_prices['CD_TYPE_NL'] == 'Alle huizen met 2, 3, 4 of meer gevels (excl. appartementen)') & (df_statbel_prices['CD_PERIOD'] == 'S1')]

In [ ]:
df_statbel_prices_houses = (
    df_statbel_prices
    .query("CD_TYPE_NL == 'Alle huizen met 2, 3, 4 of meer gevels (excl. appartementen)' & CD_PERIOD == 'S1'")  
    .loc[:, ['CD_REFNIS', 'CD_REFNIS_NL', 'MS_TOTAL_TRANSACTIONS', 'MS_P_25', 'MS_P_50_median', 'MS_P_75']]  
    .assign(F_APPARTMENT = 0)
)

In [ ]:
df_statbel_prices_appartments = (
    df_statbel_prices
    .query("CD_TYPE_NL == 'Appartementen' & CD_PERIOD == 'S1'")  
    .loc[:, ['CD_REFNIS', 'CD_REFNIS_NL', 'MS_TOTAL_TRANSACTIONS', 'MS_P_25', 'MS_P_50_median', 'MS_P_75']]  
    .assign(F_APPARTMENT = 1)
)

In [ ]:
df_statbel_prices_selection = pd.concat([df_statbel_prices_houses, df_statbel_prices_appartments], ignore_index=True)

In [ ]:
display(df_statbel_prices_selection)

In [ ]:
df_statbel_prices_selection[df_statbel_prices_selection['CD_REFNIS_NL'].str.contains('PROVINCIE')]

In [ ]:
with open('intermediate_data/statbel_prices_selection.pkl', 'wb') as file:
    pickle.dump(df_statbel_prices_selection, file)

In [ ]:
# Example of NIS where there are no appartment prices available
df_statbel_prices[df_statbel_prices['CD_REFNIS'] == 13049]

In [ ]:
df_train = pd.merge(df_train, df_statbel_prices_selection, how='left', left_on=['nis_code', 'is_appartment'], right_on = ['CD_REFNIS', 'F_APPARTMENT'])

In [ ]:
display(df_train)

In [ ]:
# Still quite some missings, whereas profiling below suggests it's a highly relevant feature

df_train['MS_P_50_median'].isna().sum()

In [ ]:
# Calculate median of MS_P_50_median per zip code and house vs. appartement for imputation

df_statbel_prices_selection = pd.merge(df_statbel_prices_selection, df_map_nis_deduplicated, how='left', left_on=['CD_REFNIS'], right_on = ['nis_code'])
display(df_statbel_prices_selection)

In [ ]:
df_statbel_prices_selection["zip_code_first2"] = df_statbel_prices_selection.zip_code.astype(str).str[:2]
median_price_per_zipcode = df_statbel_prices_selection.groupby(['zip_code_first2', 'F_APPARTMENT'], as_index = False)['MS_P_50_median'].median()
median_price_per_zipcode.columns = ['zip_code_first2', 'is_appartment', 'med_MS_P_50_median']
display(median_price_per_zipcode.head(20))

In [ ]:
median_price_per_zipcode['med_MS_P_50_median'].isna().sum()

In [ ]:
df_statbel_prices_selection[df_statbel_prices_selection['CD_REFNIS'] == 13049]

In [ ]:
df_statbel_prices[df_statbel_prices['CD_REFNIS'] == 54007]

In [ ]:
price_lookup_zipcode = median_price_per_zipcode.set_index(
    ['zip_code_first2', 'is_appartment']
)['med_MS_P_50_median'].to_dict()

In [ ]:
with open('intermediate_data/price_lookup_zipcode.pkl', 'wb') as file:
    pickle.dump(price_lookup_zipcode, file)

In [ ]:
df_train["zip_code_first2"] = df_train.zip_code.astype(str).str[:2]

df_train["MS_P_50_median_imputed"] = df_train.apply(
    lambda row: price_lookup_zipcode.get(
        (row["zip_code_first2"], row["is_appartment"]),
        row["MS_P_50_median"]
    ) if pd.isna(row["MS_P_50_median"]) else row["MS_P_50_median"],
    axis=1
)

In [ ]:
display(df_train)

In [ ]:
df_statbel_prices_selection[df_statbel_prices_selection['CD_REFNIS'] == 13049]

In [ ]:
# Still quite some missings, whereas profiling below suggests it's a highly relevant feature

df_train['MS_P_50_median_imputed'].isna().sum()

In [ ]:
df_train[df_train['MS_P_50_median_imputed'].isna()]

In [ ]:
df_statbel_prices_selection["zip_code_first1"] = df_statbel_prices_selection.zip_code.astype(str).str[:1]
median_price_per_province = df_statbel_prices_selection.groupby(['zip_code_first1', 'F_APPARTMENT'], as_index = False)['MS_P_50_median'].median()
median_price_per_province.columns = ['zip_code_first1', 'is_appartment', 'med_MS_P_50_median']
display(median_price_per_province.head(20))

In [ ]:
price_lookup_province = median_price_per_province.set_index(
    ['zip_code_first1', 'is_appartment']
)['med_MS_P_50_median'].to_dict()

In [ ]:
with open('intermediate_data/price_lookup_province.pkl', 'wb') as file:
    pickle.dump(price_lookup_province, file)

In [ ]:
df_train["zip_code_first1"] = df_train.zip_code.astype(str).str[:1]

df_train["MS_P_50_median_imputed"] = df_train.apply(
    lambda row: price_lookup_province.get(
        (row["zip_code_first1"], row["is_appartment"]),
        row["MS_P_50_median"]
    ) if pd.isna(row["MS_P_50_median_imputed"]) else row["MS_P_50_median_imputed"],
    axis=1
)

In [ ]:
df_train[df_train['MS_P_50_median_imputed'].isna()]

## Regional prices - Price per area 

## Profile again

In [ ]:
# Leave out percentiles 25 and 75 for prices per NIS code because of multicollinearity
# Leave out row count because it's not available when making a single prediction and has low predictive power (it's usually just 1)

df_train_sel = df_train[['bedrooms', 'new_building', 'foto_amount', 'province', 'subtype_regrouped', 'area_missing', 'area_imputed', 'area_rel_to_bedrooms', 'energy_value_missing', 'energy_value_imputed', 'advertiser_count', 'median_price_advertiser_bin', 'MS_TOTAL_TRANSACTIONS', 'MS_P_50_median_imputed', 'price']].drop_duplicates()

In [ ]:
profile_sel = ProfileReport(df_train_sel)

In [ ]:
profile_sel.to_notebook_iframe()

In [ ]:
# Duplicate rows?

df_train_sel.loc[170]

In [ ]:
duplicate_row_indices = [30, 32, 33]

In [ ]:
print(df_train_sel.index[:10])

In [ ]:
df_train_sel.iloc[duplicate_row_indices]

In [ ]:
df_train_sel[
    (df_train_sel["bedrooms"] == 3.0) &
    (df_train_sel["new_building"] == 1) &
    (df_train_sel["foto_amount"] == 7.0) &
    (df_train_sel["province"] == "Oost-Vlaanderen") &
    (df_train_sel["subtype_regrouped"] == "Duplex") &
    (df_train_sel["area_imputed"] == 110.0)
]

In [ ]:
df_train[
    (df_train["bedrooms"] == 3.0) &
    (df_train["new_building"] == 1) &
    (df_train["foto_amount"] == 7.0) &
    (df_train["province"] == "Oost-Vlaanderen") &
    (df_train["subtype_regrouped"] == "Duplex") &
    (df_train["area_imputed"] == 110.0)
].sort_values('added_time')

In [ ]:
df_train[(df_train['advertiser'] == 'C-Nest Groep') & (df_train['postcode'] == 9870) & (df_train['area'] == 150) & (df_train['subtype'] == 'Woning')].sort_values('price')

In [ ]:
df_train.groupby(["area", "bedrooms", "postcode", "lat", "lon", "advertiser", "foto_amount", "subtype"])['price'].nunique()

In [ ]:
df_train_sel[
    (df_train_sel["bedrooms"] == 1) &
    (df_train_sel["new_building"] == 1) &
    (df_train_sel["foto_amount"] == 25) &
    (df_train_sel["province"] == "Oost-Vlaanderen") &
    (df_train_sel["subtype_regrouped"] == "Appartement") &
    (df_train_sel["area_imputed"] == 57.1)
]

## Final variable selection and encoding

In [ ]:
display(df_train_sel)

In [ ]:
# df_train_sel = df_train_sel[df_train_sel['price'] < 500000]

In [ ]:
X_train_preprocessed = df_train_sel.drop('price', axis = 1)
X_train_preprocessed = pd.get_dummies(X_train_preprocessed, columns=['subtype_regrouped', 'province'], drop_first=True)

In [ ]:
display(X_train_preprocessed)

In [ ]:
y_train_preprocessed = df_train_sel['price']

In [ ]:
X_train_preprocessed.to_csv("split_data/train_features_preprocessed.csv", index=False)
y_train_preprocessed.to_csv("split_data/train_target_preprocessed.csv", index=False)

In [ ]:
X_train_preprocessed.columns